In [2]:
import pandas as pd
import psycopg2
import os
from openpyxl import load_workbook
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Fetch variables from the environment
hostname = os.getenv('pg_hostname')
database = os.getenv('pg_database')
username = os.getenv('pg_username')
password = os.getenv('pg_password')
port = os.getenv('pg_port')



In [3]:
cnxn_str = f"host={hostname} dbname={database} user={username} password={password} port={port}"
cnxn = psycopg2.connect(cnxn_str)

cursor = cnxn.cursor()

def sanitize_name(name):
    # Remove any character that's not alphanumeric or underscore
    name = re.sub(r'\W+', '_', name)
    # Ensure the name doesn't start with a number
    if name[0].isdigit():
        name = f"t_{name}"
    return name[:63]  # PostgreSQL has a 63-character limit for identifiers

def create_table_from_df(cursor, df, table_name):
    sanitized_table_name = sanitize_name(table_name)
    sql_create_table = f"CREATE TABLE IF NOT EXISTS {sanitized_table_name} ("
    for column in df.columns:
        sanitized_column = sanitize_name(str(column))
        sql_create_table += f"{sanitized_column} TEXT,"
    sql_create_table = sql_create_table.rstrip(',') + ');'
    cursor.execute(sql_create_table)
    cnxn.commit()

def load_df_to_sql(cursor, df, table_name):
    sanitized_table_name = sanitize_name(table_name)
    for index, row in df.iterrows():
        placeholders = ', '.join(['%s'] * len(row))
        sql_insert = f"INSERT INTO {sanitized_table_name} VALUES ({placeholders})"
        cursor.execute(sql_insert, tuple(str(val) for val in row))
    cnxn.commit()


In [4]:

folder_path = r'C:\Users\seanj\Downloads\NRS_Data'
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    try:
        if file_name.endswith('.csv'):
            df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)
            table_name = file_name.replace('.csv', '')
            create_table_from_df(cursor, df, table_name)
            load_df_to_sql(cursor, df, table_name)
        elif file_name.endswith('.xlsx'):
            xl = pd.ExcelFile(file_path)
            for sheet_name in xl.sheet_names:
                try:
                    df = xl.parse(sheet_name)
                    table_name = f"{file_name.split('.')[0]}_{sheet_name}"
                    create_table_from_df(cursor, df, table_name)
                    load_df_to_sql(cursor, df, table_name)
                except Exception as e:
                    print(f"An error occurred with sheet {sheet_name} in file {file_name}: {str(e)}")
        elif file_name.endswith('.xls'):
            try:
                xl = pd.ExcelFile(file_path)
            except ImportError:
                print(f"Unable to read {file_name}.")
                continue
            for sheet_name in xl.sheet_names:
                try:
                    df = xl.parse(sheet_name)
                    table_name = f"{file_name.split('.')[0]}_{sheet_name}"
                    create_table_from_df(cursor, df, table_name)
                    load_df_to_sql(cursor, df, table_name)
                except Exception as e:
                    print(f"An error occurred with sheet {sheet_name} in file {file_name}: {str(e)}")
    except Exception as e:
        print(f"An error occurred with file {file_name}: {str(e)}")

cursor.close()
cnxn.close()